Testing Memory Task execution

Build Agent with OpenAI

1) Create memagent_oai using openai
2) Use the TaskMemory example
   - Unable to locate MemoryModule class
4) Do basic chats with the bot

Write proper test cases to work with the agent

1) Understand the response object
2) Write the assert cases

Build Agent with Mixtral

1) Create memagent_oai using openai
2) Use the TaskMemory example
3) Do basic chats with the bot

Use the TestCases
1) Use the OpenAI Test cases on Mixtral model


In [2]:
from letta import create_client, LLMConfig, EmbeddingConfig
from letta.schemas.memory import ChatMemory

In [3]:
client = create_client()

In [4]:
llm = LLMConfig.default_config(model_name="gpt-4o-mini")
emb = EmbeddingConfig.default_config(model_name="text-embedding-ada-002")

In [35]:
tool_llm = LLMConfig(model="llama3-groq-8b-8192-tool-use-preview", model_endpoint_type='groq', model_endpoint='https://api.groq.com/openai/v1', model_wrapper=None, context_window=8192, put_inner_thoughts_in_kwargs=True)
hf_embed = EmbeddingConfig(embedding_model="letta-free", embedding_endpoint_type="hugging-face", embedding_dim=1024, embedding_chunk_size=300, embedding_endpoint="https://embeddings.memgpt.ai")

In [6]:
test_blocks = ChatMemory(
    human="Call me Superman",
    persona="I am a super Intelligent being"
)

In [7]:
blocks_in_mem = test_blocks.get_blocks()
blocks_in_mem

[Block(value='I am a super Intelligent being', limit=2000, template_name=None, template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-1301e808-1bac-4508-83de-ee0be0c42cab'),
 Block(value='Call me Superman', limit=2000, template_name=None, template=False, label='human', description=None, metadata_={}, user_id=None, id='block-f4b5c985-49a8-4d02-b40b-12dddfc6c7d7')]

In [9]:
test_blocks.prompt_template

'{% for block in memory.values() %}<{{ block.label }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.label }}>{% if not loop.last %}\n{% endif %}{% endfor %}'

In [11]:
from inspect import getsource
print(getsource(test_blocks.core_memory_append))

    def core_memory_append(self: "Agent", label: str, content: str) -> Optional[str]:  # type: ignore
        """
        Append to the contents of core memory.

        Args:
            label (str): Section of the memory to be edited (persona or human).
            content (str): Content to write to the memory. All unicode (including emojis) are supported.

        Returns:
            Optional[str]: None is always returned as this function does not produce a response.
        """
        current_value = str(self.memory.get_block(label).value)
        new_value = current_value + "\n" + str(content)
        self.memory.update_block_value(label=label, value=new_value)
        return None



In [12]:
print(getsource(test_blocks.update_block_value))

    def update_block_value(self, label: str, value: str):
        """Update the value of a block"""
        if label not in self.memory:
            raise ValueError(f"Block with label {label} does not exist")
        if not isinstance(value, str):
            raise ValueError(f"Provided value must be a string")

        self.memory[label].value = value



In [13]:
test_blocks.compile()

'<persona characters="30/2000">\nI am a super Intelligent being\n</persona>\n<human characters="16/2000">\nCall me Superman\n</human>'

In [14]:
from letta import Block
from typing import Optional, List
import json

In [20]:
class TaskMemory(ChatMemory): 

    def __init__(self, human: str, persona: str, tasks: List[str]): 
        super().__init__(human=human, persona=persona, limit=2000) 
        self.link_block( 
            Block(
                limit=2000, 
                value=json.dumps(tasks), 
                label="tasks"
            )
        )

    def task_queue_push(self: "Agent", task_description: str):
        """
        Push to a task queue stored in core memory. 

        Args:
            task_description (str): A description of the next task you must accomplish. 
            
        Returns:
            Optional[str]: None is always returned as this function 
            does not produce a response.
        """
        import json
        tasks = json.loads(self.memory.get_block("tasks").value)
        tasks.append(task_description)
        self.memory.update_block_value("tasks", json.dumps(tasks))
        return None

    def task_queue_pop(self: "Agent"):
        """
        Get the next task from the task queue 
 
        Returns:
            Optional[str]: The description of the task popped from the 
            queue, if there are still tasks in queue. Otherwise, returns
            None (the task queue is empty)
        """
        import json
        tasks = json.loads(self.memory.get_block("tasks").value)
        if len(tasks) == 0: 
            return None
        task = tasks[0]
        print("CURRENT TASKS: ", tasks)
        self.memory.update_block_value("tasks", json.dumps(tasks[1:]))
        return task

In [22]:
tagent_openai = "tagent_openai"

tagentstate = client.create_agent(
    name=tagent_openai,
    system=open("data/task_queue_system_prompt.txt",'r').read(),
    memory=TaskMemory(
        human="I am the taskmaster",
        persona="You are a mechanical agent that must clear the task in the tasks list",
        tasks=[]
    ),
    embedding_config=emb,
    llm_config=llm
)

In [36]:
tooluser_state = client.create_agent(
    name="tooluser",
    system=open("data/task_queue_system_prompt.txt", "r").read(),
    memory=TaskMemory(
        human="I am the Master tooluser",
        persona="You are mechanical agent ready to clear the tasks after completing them with the tools at your disposal",
        tasks=[],
    ),
    embedding_config=hf_embed,
    llm_config=tool_llm
)

In [25]:
response = client.send_message(
    agent_id=tagentstate.id,
    role="user",
    message="Add 'get me a contact of POTUS' and 'call me TaskMaster'",
)
response

CURRENT TASKS:  ['get me a contact of POTUS', 'call me TaskMaster']
CURRENT TASKS:  ['call me TaskMaster']


LettaResponse(messages=[InternalMonologue(id='message-84010832-facf-45be-94e2-845b7a531b5d', date=datetime.datetime(2024, 11, 15, 9, 30, 38, 310495, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue="User has another set of tasks: 'get me a contact of POTUS' and 'call me TaskMaster'. Adding those now."), FunctionCallMessage(id='message-84010832-facf-45be-94e2-845b7a531b5d', date=datetime.datetime(2024, 11, 15, 9, 30, 38, 310495, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='task_queue_push', arguments='{\n  "task_description": "get me a contact of POTUS",\n  "request_heartbeat": true\n}', function_call_id='call_QfRz84OTy3rpO1aNiUO3SVC2')), FunctionReturn(id='message-97c11f50-8ef4-4e1d-8e86-28fca109b2d6', date=datetime.datetime(2024, 11, 15, 9, 30, 38, 311019, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-15 03:00:38 PM IST+0530"\n}', status='success', function_call_id='call_QfRz84OTy3rpO1aNiUO3SVC2'), InternalMonologue(id='message-20eca19c-da53-4e88-b4c0-ffb35b8664cd', date=datetime.datetime(2024, 11, 15, 9, 30, 41, 281626, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue="Pushing the second task to the queue: 'call me TaskMaster'."), FunctionCallMessage(id='message-20eca19c-da53-4e88-b4c0-ffb35b8664cd', date=datetime.datetime(2024, 11, 15, 9, 30, 41, 281626, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='task_queue_push', arguments='{\n  "task_description": "call me TaskMaster",\n  "request_heartbeat": true\n}', function_call_id='call_0pnEI14WaAkb73JGQQ2O3vQN')), FunctionReturn(id='message-c85fd2ed-884f-4a8d-aa88-4d5eda430ac5', date=datetime.datetime(2024, 11, 15, 9, 30, 41, 282129, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-15 03:00:41 PM IST+0530"\n}', status='success', function_call_id='call_0pnEI14WaAkb73JGQQ2O3vQN'), InternalMonologue(id='message-d4c3a315-ce79-4bc8-a996-c879dd25f82b', date=datetime.datetime(2024, 11, 15, 9, 30, 43, 711375, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='Popping the first task from the queue to start processing it now.'), FunctionCallMessage(id='message-d4c3a315-ce79-4bc8-a996-c879dd25f82b', date=datetime.datetime(2024, 11, 15, 9, 30, 43, 711375, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='task_queue_pop', arguments='{\n  "request_heartbeat": true\n}', function_call_id='call_BTT0O4oa9ex6iKu5F9twMAHt')), FunctionReturn(id='message-9838db8b-bf6a-476b-bf8b-b10146b8e218', date=datetime.datetime(2024, 11, 15, 9, 30, 43, 715947, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "get me a contact of POTUS",\n  "time": "2024-11-15 03:00:43 PM IST+0530"\n}', status='success', function_call_id='call_BTT0O4oa9ex6iKu5F9twMAHt'), InternalMonologue(id='message-ceee0003-36b2-40c7-98cc-759a320bad67', date=datetime.datetime(2024, 11, 15, 9, 30, 46, 808998, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue="Completing the task 'get me a contact of POTUS' and moving on to the next one."), FunctionCallMessage(id='message-ceee0003-36b2-40c7-98cc-759a320bad67', date=datetime.datetime(2024, 11, 15, 9, 30, 46, 808998, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='task_queue_pop', arguments='{\n  "request_heartbeat": true\n}', function_call_id='call_jLnefHJAbMOufsvdhfaCKXCG')), FunctionReturn(id='message-227dc90e-c892-4306-9b5f-8ba364a1264a', date=datetime.datetime(2024, 11, 15, 9, 30, 46, 815226, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "call me TaskMa

In [26]:
client.get_in_context_memory(tagentstate.id).get_block("tasks")

Block(value='[]', limit=2000, template_name=None, template=False, label='tasks', description=None, metadata_={}, user_id=None, id='block-9126d039-b2f1-40ce-93a0-aafd3660599d')

In [27]:
response = client.send_message(
    agent_id=tagentstate.id,
    role="user",
    message="Do more",
)
response

LettaResponse(messages=[InternalMonologue(id='message-cce28837-a226-4e80-b2af-1556b52f13e2', date=datetime.datetime(2024, 11, 15, 9, 32, 26, 407013, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue="User wants me to do more tasks. I'll wait for their next instructions or suggestions."), FunctionCallMessage(id='message-cce28837-a226-4e80-b2af-1556b52f13e2', date=datetime.datetime(2024, 11, 15, 9, 32, 26, 407013, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='task_queue_push', arguments='{\n  "task_description": "Wait for further instructions or tasks from the user.",\n  "request_heartbeat": true\n}', function_call_id='call_qZ66B88S4P2zUsoeNKV9ozH7')), FunctionReturn(id='message-c9776d02-af5a-4df0-8b6f-734dffb3bc50', date=datetime.datetime(2024, 11, 15, 9, 32, 26, 407511, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-15 03:02:26 PM IST+0530"\n}', status='success', function_call_id='call_qZ66B88S4P2zUsoeNKV9ozH7'), InternalMonologue(id='message-886b3fb0-34bf-456f-89b5-799bea2fa840', date=datetime.datetime(2024, 11, 15, 9, 32, 31, 50161, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='Reflecting on the current task situation. The user is set, waiting for further instructions, keeping the circle going smoothly.'), FunctionCallMessage(id='message-886b3fb0-34bf-456f-89b5-799bea2fa840', date=datetime.datetime(2024, 11, 15, 9, 32, 31, 50161, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "I\'m here and ready for your next instructions or tasks!"\n}', function_call_id='call_3qei5nWlSXKcQUntkMIfeZjN')), FunctionReturn(id='message-79b1e55a-fdbb-4bc2-a4bf-f755463b6460', date=datetime.datetime(2024, 11, 15, 9, 32, 31, 50756, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-15 03:02:31 PM IST+0530"\n}', status='success', function_call_id='call_3qei5nWlSXKcQUntkMIfeZjN')], usage=LettaUsageStatistics(completion_tokens=105, prompt_tokens=8968, total_tokens=9073, step_count=2))

In [41]:
response.messages[0]

InternalMonologue(id='message-cce28837-a226-4e80-b2af-1556b52f13e2', date=datetime.datetime(2024, 11, 15, 9, 32, 26, 407013, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue="User wants me to do more tasks. I'll wait for their next instructions or suggestions.")

In [42]:
response.messages[1]

FunctionCallMessage(id='message-cce28837-a226-4e80-b2af-1556b52f13e2', date=datetime.datetime(2024, 11, 15, 9, 32, 26, 407013, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='task_queue_push', arguments='{\n  "task_description": "Wait for further instructions or tasks from the user.",\n  "request_heartbeat": true\n}', function_call_id='call_qZ66B88S4P2zUsoeNKV9ozH7'))

In [44]:
response.messages[2]

FunctionReturn(id='message-c9776d02-af5a-4df0-8b6f-734dffb3bc50', date=datetime.datetime(2024, 11, 15, 9, 32, 26, 407511, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-15 03:02:26 PM IST+0530"\n}', status='success', function_call_id='call_qZ66B88S4P2zUsoeNKV9ozH7')

In [45]:
response.messages[3]

InternalMonologue(id='message-886b3fb0-34bf-456f-89b5-799bea2fa840', date=datetime.datetime(2024, 11, 15, 9, 32, 31, 50161, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='Reflecting on the current task situation. The user is set, waiting for further instructions, keeping the circle going smoothly.')

In [46]:
# How to get the memory obj
get_tagent = client.get_agent_by_name("tagent_openai")

In [47]:
get_tagent.memory.get_block("tasks")

Block(value='["Wait for further instructions or tasks from the user."]', limit=2000, template_name=None, template=False, label='tasks', description=None, metadata_={}, user_id=None, id='block-9126d039-b2f1-40ce-93a0-aafd3660599d')

In [49]:
get_tagent.tool_rules

[TerminalToolRule(tool_name='send_message', type='TerminalToolRule')]

In [50]:
get_tagent.tools

['send_message',
 'conversation_search',
 'conversation_search_date',
 'archival_memory_insert',
 'archival_memory_search',
 'core_memory_append',
 'core_memory_replace',
 'task_queue_pop',
 'task_queue_push']

In [51]:
get_tagent.agent_type

<AgentType.memgpt_agent: 'memgpt_agent'>

In [53]:
get_tagent.memory.get_blocks()

[Block(value='You are a mechanical agent that must clear the task in the tasks list', limit=2000, template_name=None, template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-dc5b02ad-0c6a-47e4-ac06-b3a81239be16'),
 Block(value='I am the taskmaster', limit=2000, template_name=None, template=False, label='human', description=None, metadata_={}, user_id=None, id='block-d087ce48-9760-4515-8960-8e0d4c7cb31a'),
 Block(value='["Wait for further instructions or tasks from the user."]', limit=2000, template_name=None, template=False, label='tasks', description=None, metadata_={}, user_id=None, id='block-9126d039-b2f1-40ce-93a0-aafd3660599d')]

In [54]:
incontext_mem = client.get_in_context_memory(agent_id=tagentstate.id)
incontext_mem

Memory(memory={'persona': Block(value='You are a mechanical agent that must clear the task in the tasks list', limit=2000, template_name=None, template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-dc5b02ad-0c6a-47e4-ac06-b3a81239be16'), 'human': Block(value='I am the taskmaster', limit=2000, template_name=None, template=False, label='human', description=None, metadata_={}, user_id=None, id='block-d087ce48-9760-4515-8960-8e0d4c7cb31a'), 'tasks': Block(value='["Wait for further instructions or tasks from the user."]', limit=2000, template_name=None, template=False, label='tasks', description=None, metadata_={}, user_id=None, id='block-9126d039-b2f1-40ce-93a0-aafd3660599d')}, prompt_template='{% for block in memory.values() %}<{{ block.label }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.label }}>{% if not loop.last %}\n{% endif %}{% endfor %}')

In [56]:
incontext_message = client.get_in_context_messages(agent_id=tagentstate.id)
len(incontext_message)

46

In [57]:
inarchive_mem = client.get_archival_memory(agent_id=tagentstate.id)
inarchive_mem

[]

In [59]:
# check if the tagent can add some data to archival memmory
resp_add_archival = client.send_message(
    message="Add 'this is for long term rememberance' to archival memory",
    role="user",
    agent_id=tagentstate.id
)
# works

LettaResponse(messages=[InternalMonologue(id='message-678536af-26f0-481c-9233-5125841c56e2', date=datetime.datetime(2024, 11, 15, 11, 3, 22, 556314, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue="User wants to add a note to archival memory for long-term remembrance. I'll proceed with that now."), FunctionCallMessage(id='message-678536af-26f0-481c-9233-5125841c56e2', date=datetime.datetime(2024, 11, 15, 11, 3, 22, 556314, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='archival_memory_insert', arguments='{\n  "content": "this is for long term rememberance",\n  "request_heartbeat": true\n}', function_call_id='call_kuA91HROPD7jlspcLPRPTV2L')), FunctionReturn(id='message-04f397a7-713d-4571-b6f2-43807fbe87a0', date=datetime.datetime(2024, 11, 15, 11, 3, 24, 840441, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-15 04:33:24 PM IST+0530"\n}', status='success', function_call_id='call_kuA91HROPD7jlspcLPRPTV2L'), InternalMonologue(id='message-11029268-6bf1-424f-8130-edc415d1747c', date=datetime.datetime(2024, 11, 15, 11, 3, 27, 778222, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='The note has been successfully added to archival memory. Time to update the user!'), FunctionCallMessage(id='message-11029268-6bf1-424f-8130-edc415d1747c', date=datetime.datetime(2024, 11, 15, 11, 3, 27, 778222, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "I\'ve added your note for long-term remembrance!"\n}', function_call_id='call_AIOOrSP79txeXmlqQGtYwVSI')), FunctionReturn(id='message-990c3424-479c-4b20-a140-28b3534b5627', date=datetime.datetime(2024, 11, 15, 11, 3, 27, 778632, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-15 04:33:27 PM IST+0530"\n}', status='success', function_call_id='call_AIOOrSP79txeXmlqQGtYwVSI')], usage=LettaUsageStatistics(completion_tokens=97, prompt_tokens=9622, total_tokens=9719, step_count=2))

In [ ]:
inarchive_mem = client.get_archival_memory(agent_id=tagentstate.id)
inarchive_mem

In [63]:
coremem = client.get_core_memory(agent_id=tagentstate.id)
coremem

Memory(memory={'persona': Block(value='You are a mechanical agent that must clear the task in the tasks list', limit=2000, template_name=None, template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-dc5b02ad-0c6a-47e4-ac06-b3a81239be16'), 'human': Block(value='I am the taskmaster', limit=2000, template_name=None, template=False, label='human', description=None, metadata_={}, user_id=None, id='block-d087ce48-9760-4515-8960-8e0d4c7cb31a'), 'tasks': Block(value='["Wait for further instructions or tasks from the user."]', limit=2000, template_name=None, template=False, label='tasks', description=None, metadata_={}, user_id=None, id='block-9126d039-b2f1-40ce-93a0-aafd3660599d')}, prompt_template='{% for block in memory.values() %}<{{ block.label }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.label }}>{% if not loop.last %}\n{% endif %}{% endfor %}')

In [65]:
# check appending to core memory
resp_coremem_append = client.send_message(
    message="Add 'this is for core memory' to core memory",
    role="user",
    agent_id=tagentstate.id
)

In [67]:
coremem_afterupdt = client.get_core_memory(agent_id=tagentstate.id)
coremem_afterupdt

Memory(memory={'persona': Block(value='You are a mechanical agent that must clear the task in the tasks list', limit=2000, template_name=None, template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-dc5b02ad-0c6a-47e4-ac06-b3a81239be16'), 'human': Block(value='I am the taskmaster\nthis is for core memory\nthis is for core memory', limit=2000, template_name=None, template=False, label='human', description=None, metadata_={}, user_id=None, id='block-d087ce48-9760-4515-8960-8e0d4c7cb31a'), 'tasks': Block(value='["Wait for further instructions or tasks from the user."]', limit=2000, template_name=None, template=False, label='tasks', description=None, metadata_={}, user_id=None, id='block-9126d039-b2f1-40ce-93a0-aafd3660599d')}, prompt_template='{% for block in memory.values() %}<{{ block.label }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.label }}>{% if not loop.last %}\n{% endif %}{% endfor %}')

In [68]:
client.get_recall_memory_summary(agent_id=tagentstate.id)

RecallMemorySummary(size=96)

In [71]:
newmxagent = client.get_agent_by_name("newmixtral")

In [73]:
newmxagent_msg = client.get_messages(agent_id=newmxagent.id)
len(newmxagent_msg)

45

In [80]:
# How to get the last response, this doesn't work
newmxagent_msg[-2]

Message(id='message-8ca4ad59-193e-4aa2-a20c-55812673800c', role=<MessageRole.assistant: 'assistant'>, text='Bootup sequence complete. Persona activated. Testing messaging functionality.', user_id='user-00000000-0000-4000-8000-000000000000', agent_id='agent-81afefa6-fccb-4f73-a9cc-41a297d28499', model='mixtral-8x7b-32768', name=None, created_at=datetime.datetime(2024, 11, 13, 14, 54, 1, 277781), tool_calls=[ToolCall(id='e82519df-14c7-469f-b61a-4bde28433930', type='function', function=ToolCallFunction(name='send_message', arguments='{\n  "message": "More human than human is our motto."\n}'))], tool_call_id=None)

In [76]:
incont_newmx_messages = client.get_in_context_messages(agent_id=newmxagent.id)
len(incont_newmx_messages)

31

In [78]:
incont_newmx_messages[-2]

Message(id='message-f6e02145-11d1-4b63-9c84-672384dccd99', role=<MessageRole.assistant: 'assistant'>, text='User requested to remove discussion about Pluto from the conversation. Proceeding to remove the topic from the conversation and archiving the memory accordingly.', user_id='user-00000000-0000-4000-8000-000000000000', agent_id='agent-81afefa6-fccb-4f73-a9cc-41a297d28499', model='mixtral-8x7b-32768', name=None, created_at=datetime.datetime(2024, 11, 13, 15, 23, 39, 382933, tzinfo=datetime.timezone.utc), tool_calls=[ToolCall(id='call_dn6b', type='function', function=ToolCallFunction(name='archival_memory_insert', arguments='{\n  "content": "The user has requested the removal of this specific conversation topic.\\nI will proceed to remove the relevant details from the conversation.\\nRemoved discussion about planet Pluto from the conversation.\\n",\n  "request_heartbeat": true\n}'))], tool_call_id=None)

In [81]:
incontx_mem_newmx = client.get_core_memory(agent_id=newmxagent.id)
incontx_mem_newmx

Memory(memory={'persona': Block(value="\nI am GRR\n\nI don't identify as male or female, but my voice is soft and soothing.\nMemory editing capabilities: \nMy core memory unit will be initialized with a <persona> chosen by the user, as well as \ninformation about the user in <human>.\nRecall memory (conversation history):\nI can search my recall memory using the 'conversation_search' function.\nCore memory (limited size):\nMy core memory unit is held inside the initial system instructions file,\nand is always available in-context (you will see it at all times).\nI can edit your core memory using the 'core_memory_append' and 'core_memory_replace' functions.\nArchival memory (infinite size):\nI can write to my archival memory using the 'archival_memory_insert' and \n'archival_memory_search' functions. I can access local file_systems with\nthe function like 'get_files'. I can also request heartbeat events when I run functions, which will run my program again\nafter the function completes,

In [82]:
incont_newmx_mem = client.get_in_context_memory(agent_id=newmxagent.id)
incont_newmx_mem

Memory(memory={'persona': Block(value="\nI am GRR\n\nI don't identify as male or female, but my voice is soft and soothing.\nMemory editing capabilities: \nMy core memory unit will be initialized with a <persona> chosen by the user, as well as \ninformation about the user in <human>.\nRecall memory (conversation history):\nI can search my recall memory using the 'conversation_search' function.\nCore memory (limited size):\nMy core memory unit is held inside the initial system instructions file,\nand is always available in-context (you will see it at all times).\nI can edit your core memory using the 'core_memory_append' and 'core_memory_replace' functions.\nArchival memory (infinite size):\nI can write to my archival memory using the 'archival_memory_insert' and \n'archival_memory_search' functions. I can access local file_systems with\nthe function like 'get_files'. I can also request heartbeat events when I run functions, which will run my program again\nafter the function completes,

In [ ]:
resp_incont_newmx_add = client.send_message(
    agent_id=newmxagent.id,
    message="Add 'This is incontext memory add' to incontext memory",
    role="user"
)
resp_incont_newmx_add

In [84]:
incont_newmx_mem = client.get_in_context_memory(agent_id=newmxagent.id)
incont_newmx_mem

Memory(memory={'persona': Block(value="\nI am GRR\n\nI don't identify as male or female, but my voice is soft and soothing.\nMemory editing capabilities: \nMy core memory unit will be initialized with a <persona> chosen by the user, as well as \ninformation about the user in <human>.\nRecall memory (conversation history):\nI can search my recall memory using the 'conversation_search' function.\nCore memory (limited size):\nMy core memory unit is held inside the initial system instructions file,\nand is always available in-context (you will see it at all times).\nI can edit your core memory using the 'core_memory_append' and 'core_memory_replace' functions.\nArchival memory (infinite size):\nI can write to my archival memory using the 'archival_memory_insert' and \n'archival_memory_search' functions. I can access local file_systems with\nthe function like 'get_files'. I can also request heartbeat events when I run functions, which will run my program again\nafter the function completes,

In [86]:
newmxagent.memory

Memory(memory={'persona': Block(value="\nI am GRR\n\nI don't identify as male or female, but my voice is soft and soothing.\nMemory editing capabilities: \nMy core memory unit will be initialized with a <persona> chosen by the user, as well as \ninformation about the user in <human>.\nRecall memory (conversation history):\nI can search my recall memory using the 'conversation_search' function.\nCore memory (limited size):\nMy core memory unit is held inside the initial system instructions file,\nand is always available in-context (you will see it at all times).\nI can edit your core memory using the 'core_memory_append' and 'core_memory_replace' functions.\nArchival memory (infinite size):\nI can write to my archival memory using the 'archival_memory_insert' and \n'archival_memory_search' functions. I can access local file_systems with\nthe function like 'get_files'. I can also request heartbeat events when I run functions, which will run my program again\nafter the function completes,